In [148]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import joblib
import time
from datetime import datetime



from sklearn.preprocessing import MinMaxScaler , PolynomialFeatures
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [149]:
# 넘어온 정보를 가중치와 병합, 길이를 맞춤
def pre_predict(wea,gu):
    weather = pd.DataFrame()
    station_W = pd.read_csv('./data/station_W.csv',encoding='cp949')

    W = station_W[station_W['GU'] == gu].set_index('STATION_NAME',drop=False)[['W']]
    wea = pd.DataFrame(wea)
    W = np.log1p(W)
    # gu에서 받아온 역의 갯수만큼 날씨변수 생성
    for i in range(len(W)):
        weather = pd.concat([weather,wea],axis=0)
        # gu에서 받아온 역의 갯수만큼 날씨변수 생성
    weather.index = W.index
    feat=weather.columns.drop('ONDO')
    weather[feat]=np.log1p(weather[feat])
    values = pd.concat([weather,W], axis=1)
    values.index = W.index# 날씨와 가중치 컬럼 합병
    print(values)
    return values

In [160]:
# 웹서버에 모델 저장해놓기(csv와 경로 똑같이)
def predict(values):
    lr = joblib.load('weater_lr.pkl')
    
    coef=print(lr.intercept_.round(3),lr.coef_.round(3))
    result = lr.predict(values)
    result = pd.DataFrame(np.expm1(result))
    result.index = values.index
    result.columns = ['result']
    result['result']=result['result'].astype(int)
    result.sort_values(by='result', ascending=False, inplace=True)
    result.reset_index(drop=False,inplace=True)

    return result

In [161]:
# 0. view.py에서 weather_import() 호출
def weather_import(day,time,gu):
# 3일치를 모두 csv파일로 저장
# csv 로드 후 day가 key
# 1. csv 로드 3일치 예보
    wea = pd.read_csv('data/forecast.csv',encoding='cp949')# 1. csv 로드 3일치 예보
# 2. 로드된 df에서 오늘/내일/모레 & 시간를 인덱싱 후 가져오기
    wea = wea[['RAIN', 'HUMN', 'SNOW', 'SKY', 'ONDO', 'WINDD', 'WINDS', 'PM10', 'PM25', 'DAY', 'TIME']] # 컬럼 순서에 맞게 정렬
    T=[0 if (int(i.split(':')[0])>19)|(int(i.split(':')[0])<17) else 1 for i in wea['TIME']] # 시간대 컬럼 생성
    wea['TIME'] = pd.to_datetime(wea['TIME']) # 시간정보를 문자열에서 시간변수로 변환
    wea['time'] = wea['TIME'].dt.strftime('%H').astype(int)# 변환된 시간변수로 시간컬럼 생성
    wea.drop(['DAY', 'TIME'],axis=1, inplace=True)# 불필요 컬럼 제거
#     wea['PM10'] = [32.8]*39# 미세먼지 컬럼 (임시)
#     wea['PM25'] = [19.2]*39# 초미세먼지 컬럼 (임시)
    wea['T'] = T # 시간대 컬럼 삽입
    wea = {'today':wea[:13], 'day1':wea[13:26],'day2':wea[26:]} # 3일치에 맞게 컬럼 분할
    #print(wea)
    weather = wea[day][wea[day]['time'] == int(time)] # 요청받은 날짜, 시간에 맞는 예보데이터 인덱싱
    #print(weather)
    weather=weather[['RAIN', 'HUMN', 'SNOW', 'SKY', 'ONDO', 'WINDD', 'WINDS','PM10','PM25','T']]
    #     weather.drop('time',axis=1,inplace=True) # 인덱싱을 위한 시간변수 삭제

# 3. 예측
    result = predict(pre_predict(weather,gu))
    result.index=result.index+1
    return result,weather


# (py파일, 모델파일, 날씨.csv, station_W.csv 같은 폴더에)

In [162]:
result,weather=weather_import('day1', '21', '서초구')

              RAIN      HUMN  SNOW       SKY  ONDO     WINDD     WINDS  \
STATION_NAME                                                             
고속터미널          0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
교대             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
남부터미널          0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
남태령            0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
내방             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
반포             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
방배             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
서초             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
양재             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   
잠원             0.0  4.394449   0.0  0.693147    10  1.098612  0.262364   

                  PM10      PM25    T         W  
STATION_NAME                                     
고속터미널      

In [146]:
weather

,RAIN,HUMN,SNOW,SKY,ONDO,WINDD,WINDS,PM10,PM25,T
24,0,80,0,1,10,2,0.3,55,55,0


In [144]:
result

,STATION_NAME,result
1,고속터미널,4274
2,교대,2840
3,양재,2146
4,남부터미널,1797
5,서초,1248
6,방배,1135
7,내방,755
8,잠원,516
9,반포,422
10,남태령,243


In [102]:
result['result']=result['result'].astype(int)

In [103]:
result

,STATION_NAME,result
1,고속터미널,4135
2,교대,2748
3,양재,2076
4,남부터미널,1739
5,서초,1207
6,방배,1098
7,내방,730
8,잠원,499
9,반포,408
10,남태령,235


In [104]:
result=result.reset_index()

In [106]:
result.to_dict('records')

[{'index': 1, 'STATION_NAME': '고속터미널', 'result': 4135},
 {'index': 2, 'STATION_NAME': '교대', 'result': 2748},
 {'index': 3, 'STATION_NAME': '양재', 'result': 2076},
 {'index': 4, 'STATION_NAME': '남부터미널', 'result': 1739},
 {'index': 5, 'STATION_NAME': '서초', 'result': 1207},
 {'index': 6, 'STATION_NAME': '방배', 'result': 1098},
 {'index': 7, 'STATION_NAME': '내방', 'result': 730},
 {'index': 8, 'STATION_NAME': '잠원', 'result': 499},
 {'index': 9, 'STATION_NAME': '반포', 'result': 408},
 {'index': 10, 'STATION_NAME': '남태령', 'result': 235}]

In [118]:
wea = pd.read_csv('data/forecast.csv',encoding='cp949')

In [120]:
feat=wea.columns.drop(['ONDO','DAY','TIME'])

In [121]:
wea[feat]

,RAIN,HUMN,SNOW,SKY,WINDD,WINDS,PM10,PM25
0,0,60,0,4,7,1.0,55,25
1,0,50,0,3,8,0.7,55,25
2,0,50,0,3,7,1.0,55,25
3,0,45,0,4,10,1.7,55,25
4,0,45,0,3,10,1.9,55,25
5,0,45,0,4,11,2.1,55,25
6,0,50,0,4,11,2.1,55,25
7,0,55,0,4,11,1.3,55,25
8,0,65,0,4,10,1.3,55,25
9,0,70,0,4,10,1.3,55,25


In [123]:
wea[feat]=wea[feat].apply(np.log1p)

In [124]:
wea

,DAY,TIME,RAIN,HUMN,SNOW,SKY,ONDO,WINDD,WINDS,PM10,PM25
0,2022-11-07,10:00,0.0,4.110874,0.0,1.609438,11,2.079442,0.693147,4.025352,3.258097
1,2022-11-07,11:00,0.0,3.931826,0.0,1.386294,13,2.197225,0.530628,4.025352,3.258097
2,2022-11-07,12:00,0.0,3.931826,0.0,1.386294,14,2.079442,0.693147,4.025352,3.258097
3,2022-11-07,13:00,0.0,3.828641,0.0,1.609438,15,2.397895,0.993252,4.025352,3.258097
4,2022-11-07,14:00,0.0,3.828641,0.0,1.386294,15,2.397895,1.064711,4.025352,3.258097
5,2022-11-07,15:00,0.0,3.828641,0.0,1.609438,15,2.484907,1.131402,4.025352,3.258097
6,2022-11-07,16:00,0.0,3.931826,0.0,1.609438,15,2.484907,1.131402,4.025352,3.258097
7,2022-11-07,17:00,0.0,4.025352,0.0,1.609438,14,2.484907,0.832909,4.025352,3.258097
8,2022-11-07,18:00,0.0,4.189655,0.0,1.609438,13,2.397895,0.832909,4.025352,3.258097
9,2022-11-07,19:00,0.0,4.262680,0.0,1.609438,12,2.397895,0.832909,4.025352,3.258097
